In [2]:
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 68.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 6.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.0/112.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 MB 37.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:000:00:0100

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 
dtype = None 
load_in_4bit = True 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-0.5B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.3.8: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [84]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("json", data_files="../input/20-question/train_sharegpt.json", split = "train")

In [85]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True)

In [86]:
dataset[5]["conversations"]

[{'content': "I'm thinking of an object. Try to guess it by asking yes/no questions. (The object is: Bush)",
  'role': 'user'},
 {'content': 'Is the object living?', 'role': 'assistant'},
 {'content': 'Yes.', 'role': 'user'},
 {'content': 'Is the object a mammal?', 'role': 'assistant'},
 {'content': 'No.', 'role': 'user'},
 {'content': 'Is the object an inanimate object?', 'role': 'assistant'},
 {'content': 'Yes.', 'role': 'user'},
 {'content': 'Is the object man-made?', 'role': 'assistant'},
 {'content': 'No.', 'role': 'user'},
 {'content': 'Is the object a natural object?', 'role': 'assistant'},
 {'content': 'Yes.', 'role': 'user'},
 {'content': 'Is the object a mineral?', 'role': 'assistant'},
 {'content': 'No.', 'role': 'user'},
 {'content': 'Is the object a plant?', 'role': 'assistant'},
 {'content': 'Yes.', 'role': 'user'},
 {'content': 'Is the object a tree?', 'role': 'assistant'},
 {'content': 'No.', 'role': 'user'},
 {'content': 'Is the object a flower?', 'role': 'assistant'},

In [87]:
dataset[5]["text"]

"<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nI'm thinking of an object. Try to guess it by asking yes/no questions. (The object is: Bush)<|im_end|>\n<|im_start|>assistant\nIs the object living?<|im_end|>\n<|im_start|>user\nYes.<|im_end|>\n<|im_start|>assistant\nIs the object a mammal?<|im_end|>\n<|im_start|>user\nNo.<|im_end|>\n<|im_start|>assistant\nIs the object an inanimate object?<|im_end|>\n<|im_start|>user\nYes.<|im_end|>\n<|im_start|>assistant\nIs the object man-made?<|im_end|>\n<|im_start|>user\nNo.<|im_end|>\n<|im_start|>assistant\nIs the object a natural object?<|im_end|>\n<|im_start|>user\nYes.<|im_end|>\n<|im_start|>assistant\nIs the object a mineral?<|im_end|>\n<|im_start|>user\nNo.<|im_end|>\n<|im_start|>assistant\nIs the object a plant?<|im_end|>\n<|im_start|>user\nYes.<|im_end|>\n<|im_start|>assistant\nIs the object a tree?<|im_end|>\n<|im_start|>user\nNo.<|im_end|>\n<|im_start|>assistant\nIs the 

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 1000,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

In [89]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

In [90]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

"<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nI'm thinking of an object. Try to guess it by asking yes/no questions. (The object is: Bush)<|im_end|>\n<|im_start|>assistant\nIs the object living?<|im_end|>\n<|im_start|>user\nYes.<|im_end|>\n<|im_start|>assistant\nIs the object a mammal?<|im_end|>\n<|im_start|>user\nNo.<|im_end|>\n<|im_start|>assistant\nIs the object an inanimate object?<|im_end|>\n<|im_start|>user\nYes.<|im_end|>\n<|im_start|>assistant\nIs the object man-made?<|im_end|>\n<|im_start|>user\nNo.<|im_end|>\n<|im_start|>assistant\nIs the object a natural object?<|im_end|>\n<|im_start|>user\nYes.<|im_end|>\n<|im_start|>assistant\nIs the object a mineral?<|im_end|>\n<|im_start|>user\nNo.<|im_end|>\n<|im_start|>assistant\nIs the object a plant?<|im_end|>\n<|im_start|>user\nYes.<|im_end|>\n<|im_start|>assistant\nIs the object a tree?<|im_end|>\n<|im_start|>user\nNo.<|im_end|>\n<|im_start|>assistant\nIs the 

In [91]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                    \nIs the object living?<|im_end|>\n         \nIs the object a mammal?<|im_end|>\n         \nIs the object an inanimate object?<|im_end|>\n         \nIs the object man-made?<|im_end|>\n         \nIs the object a natural object?<|im_end|>\n         \nIs the object a mineral?<|im_end|>\n         \nIs the object a plant?<|im_end|>\n         \nIs the object a tree?<|im_end|>\n         \nIs the object a flower?<|im_end|>\n         \nIs the object a vegetable?<|im_end|>\n         \nIs the object a shrub?<|im_end|>\n         \nIs the object a climber?<|im_end|>\n         \nIs the object a herbaceous plant?<|im_end|>\n         \nIs the object an ornamental plant?<|im_end|>\n         \nIs the object a shrub like a cherry tree?<|im_end|>\n         \nIs the object a shrub like a grapevine?<|im_end|>\n         \nIs the object a shrub like a mulberry tree?<|im_end|>\n         \nIs the object a shrub like a fig tree?<|im_end|>\n         \nIs the ob

In [92]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 8,798,208/345,364,352 (2.55% trained)


Step,Training Loss
1,1.912800
2,1.902500
3,1.864800
4,1.784100
5,1.743000
6,1.674500
7,1.594600
8,1.528800
9,1.466100
10,1.525900


In [93]:
model.save_pretrained("sft_model")
tokenizer.save_pretrained("sft_model")

('sft_model/tokenizer_config.json',
 'sft_model/special_tokens_map.json',
 'sft_model/vocab.json',
 'sft_model/merges.txt',
 'sft_model/added_tokens.json',
 'sft_model/tokenizer.json')